Team ID-3, 
Sem-5, 
Div-E,
Department-Computer Science.

ACP08-Cybersecuirity Data Mining Competition.

Problem Statement-

1.Classifying attacks as Malicious or Benign.

2.Catagorizing the attacks as Catagories.

3.Classifyiing

Team Leader-Sanjay Bhat

Team Members:

Raghvendra Hallyal-01fe19bcs304

Sanjay Bhat-01fe19bcs283

Naganand Narve-01fe19bcs284

Chirag tandel-01fe19bcs280


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Mounting Google Drive onto Colab

In [21]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


Reading the Training data from CSV file and assigning Column names to it

In [22]:
columns = ['Index','Category','word_vector','Label']
train = pd.read_csv("/content/gdrive/MyDrive/DMA Data Set/Task_1/Training.tsv",sep='\t',names = columns, header=0)
train.head()

,Index,Category,word_vector,Label
0,0,24,"[[-0.507, -0.49518, 0.46885, 0.54524, -0.11552...",2.0
1,1,13,"[[0.19911, -0.46156, 0.19674, -1.3298, 0.51805...",1.0
2,2,24,"[[-0.7403, -0.78746, 0.47018, 0.43474, 0.05842...",1.0
3,3,21,"[[-0.44257, -0.54624, 0.25403, 0.80731, 1.026,...",1.0
4,4,13,"[[0.19911, -0.46156, 0.19674, -1.3298, 0.51805...",2.0


In [23]:
type(train.Label[0])

numpy.float64

In [24]:
len(train)

123040

Printing the percentage of Empty labels present in the Label cloumn

In [25]:
print(train.isna().sum())
# We can see that there are no Null values available in the Training Dataset

Index          0
Category       0
word_vector    0
Label          0
dtype: int64


Droping Index attribute and restting the indexes.


In [26]:
train.reset_index(inplace=True)
train.drop(['index','Index'],axis='columns', inplace=True)
print(train.shape[0])
train.head()

123040


,Category,word_vector,Label
0,24,"[[-0.507, -0.49518, 0.46885, 0.54524, -0.11552...",2.0
1,13,"[[0.19911, -0.46156, 0.19674, -1.3298, 0.51805...",1.0
2,24,"[[-0.7403, -0.78746, 0.47018, 0.43474, 0.05842...",1.0
3,21,"[[-0.44257, -0.54624, 0.25403, 0.80731, 1.026,...",1.0
4,13,"[[0.19911, -0.46156, 0.19674, -1.3298, 0.51805...",2.0


Since all the word vectors were in the form of strings,
Converting the Word Vector to list of lists, and making them as 2 dimensional Numpy array and reassigning them to the word vector.

In [27]:
train.word_vector = [np.mean(eval(train.iloc[i,1]),axis=1) for i in range(123040)] #estimated time 2:30 mins

Creating 2 seperate Dataframe objects, where

df1-  will have Training Dataset with mean of Word Vector

df2 - will have Training Dataset with mean of Word Vectors and One-Hot Encoded Catagory values

In [28]:
df1 = train.copy() # Training Dataset with Mean of word Vector
df2 = train.copy() #Catagory Hot-Encoded dataset with Mean of Word Vectors

def convert_to_binary(df, column_to_convert):
  categories = list(df[column_to_convert].drop_duplicates())

  for category in categories:
    cat_name = str(category)
    col_name = str(column_to_convert) + '_' + cat_name
    df[col_name] = 0
    df.loc[(df[column_to_convert] == category), col_name] = 1

  return df
df2 = convert_to_binary(df=df2, column_to_convert='Category')
df2.drop('Category', axis=1, inplace=True)

In [29]:
from tqdm import tqdm,trange

Creating 2 dataframes with overall mean of word vectors

In [30]:
row_count = []
df3 = df1.copy() #Dataframe with overall mean of word vector
df4 = df2.copy() #Dataframe with overall mean of word vector and Hot-Encoded Catagory values
for i in tqdm(range(123040),desc="Progress of Calculating mean:"):
  row_count.append(len(df3.word_vector[i]))
  df3.word_vector[i] = np.mean(df3.word_vector[i])
  df4.word_vector[i] = np.mean(df4.word_vector[i])
df3["row_count"] = row_count
df4["row_count"] = row_count
del row_count

Progress of Calculating mean::   0%|          | 0/123040 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
Progress of Calculating mean:: 100%|██████████| 123040/123040 [05:20<00:00, 383.99it/s]


Label Encoding the Label attribute and dropping the Label attribute from training Dataset

In [31]:
from sklearn.preprocessing import LabelEncoder

labels = df3['Label']

# Label encoding for Malware Label Column
le = LabelEncoder()
Y = le.fit_transform(labels)

X = df3.drop('Label', axis=1, inplace=False)

Splitting the Training to 80% training data and 20% validation data

In [32]:
#from sklearn import cross_validation
from sklearn.model_selection import train_test_split
## Spliting of training dataset into 80% training data and 20% testing data randomly
features_train, features_test, labels_train, labels_test = train_test_split(X, Y, test_size=0.2, random_state=1)

Using decision tree to obtain a model from Training data

In [33]:
# features_train.info()
## Decision Tree 
from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=50, max_depth = 5, criterion = 'entropy')
## min_samples_split=50 max_depth = 5 criterion='entropy'
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
from sklearn.metrics import accuracy_score
print (accuracy_score(prediction, labels_test)*100,'%')

72.99252275682704 %


Using Gaussian Naive Bayes

In [34]:
## Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)

## Computing accuracy
print (accuracy_score(prediction, labels_test)*100,'%')

55.76235370611183 %


Decision Tree with GridSearchCV

In [35]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV
parameters = {'criterion':('gini', 'entropy'), 'max_depth':[1, 3, 5, 10]}
dt = tree.DecisionTreeClassifier()
clf = GridSearchCV(dt, parameters)

clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)

## Computing accuracy
from sklearn.metrics import accuracy_score
print (accuracy_score(prediction, labels_test)*100,'%')

78.43790637191157 %


XGBoost Classifier

In [36]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(np.array(features_train), np.array(labels_train))
prediction = clf.predict(np.array(features_test))
print (accuracy_score(prediction, labels_test)*100,'%')

77.56014304291287 %


Gradient Boositng Classifier

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(features_train, labels_train)
prediction = clf.predict(features_test)
from sklearn.metrics import accuracy_score
print (accuracy_score(prediction, labels_test)*100,'%')

75.10159297789338 %


Extra Trees Classifier

In [38]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100,random_state=0)
clf.fit(features_train,labels_train)
prediction = clf.predict(features_test)
print (accuracy_score(prediction, labels_test)*100,'%')

79.32786085825748 %


In [39]:
test_perc_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
!pip install eli5
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_log_error
import eli5
from tqdm import tqdm_notebook
from sklearn.model_selection import KFold

     |████████████████████████████████| 106 kB 5.7 MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [40]:
estimator=ExtraTreesClassifier(n_estimators=100,random_state=0)
cv=5
n_jobs=-1
train_sizes, train_scores, test_scores = learning_curve(estimator,X, Y, cv=cv, n_jobs=n_jobs, train_sizes=test_perc_list)

In [41]:
def get_mean(arr):
    final= []
    for i in range(arr.shape[0]):
        final.append(arr[i].mean())
    return final

In [42]:
trainFinal,testFinal = get_mean(train_scores),get_mean(test_scores)
import plotly.graph_objects as go

In [43]:
fig=go.Figure()
x,y,z = test_perc_list,trainFinal,testFinal
fig.add_trace(go.Scatter(x=x , y=y , name = 'Train Score',line_shape = 'linear'))
fig.add_trace(go.Scatter(x=x , y=z , name = 'Test Score',line_shape = 'linear'))
fig.update_layout(title_text = 'Bais Variance TradeOff')
fig.update_xaxes(title_text='Train Size' , showgrid = False)
fig.update_yaxes(title_text='Accuracy' , showgrid = False)
fig.show()